In [46]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from bs4 import BeautifulSoup

from sklearn import tree
from colorama import Fore  #Colorama is a module to color the python outputs
from urllib.parse import urlparse

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from tld import get_tld, is_tld

# 現在我們可以建立我們的模型。我們將使用一個簡單的LSTM模型。
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
file = open('./100070.html','r')
content = file.read()
print(content)

<!DOCTYPE html><html data-cast-api-enabled="true" lang="zh-HK"><head><script nonce="4OZ9pLGHYRD9Idi2Q2kSEQ">var DOCS_timing={}; DOCS_timing['pls']=new Date().getTime();</script><meta property="og:title" content="Untitled presentation"><meta property="og:type" content="article"><meta property="og:site_name" content="Google Docs"><meta property="og:url" content="https://docs.google.com/presentation/d/e/2PACX-1vSMbU9SwZxIKbEB8T5xNXENzxZ74kNm9ZxeYeDmlf1wh-tDhXx3y5zggMlZtimBXmjEc2W5vVmIZ9Tk/pub?start=false&amp;amp;loop=false&amp;amp;delayms=3000&amp;usp=embed_facebook"><meta property="og:description" content="AT&amp;T YAHOO CLOSURE Click Here To Update"><meta name="google" content="notranslate"><meta name="viewport" content="target-densitydpi=device-dpi,user-scalable=1,minimum-scale=1,maximum-scale=2.5,initial-scale=1"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><meta name="fragment" content="!"><meta name="referrer" content="origin"><title>Untitled presentation - Google 簡報</title

In [39]:
hostname = 'https://docs.google.com/presentation/d/e/2PACX-1vSMbU9SwZxIKbEB8T5xNXENzxZ74kNm9ZxeYeDmlf1wh-tDhXx3y5zggMlZtimBXmjEc2W5vVmIZ9Tk/pub?start=false&amp;loop=false&amp;delayms=3000'

In [47]:
soup = BeautifulSoup(content, 'html.parser')

In [40]:
def check_anchor_url():
    regex_pattern = rf'<a.*? href="(?:https?://)?(?!.*?{re.escape(hostname)}).*?>(.*?)</a>'
    all_anchor_tag = len(re.findall(r'<a.*?>(.*?)</a>', content))
    anchor_not_linked = len(re.findall(r'<a.*?>(.*?)</a>', content))
    count_percent = anchor_not_linked / all_anchor_tag if all_anchor_tag != 0 else 1

    if count_percent < 0.31:
        return 1
    elif 0.31 <= count_percent <= 0.67:
        return 0
    else:
        return -1

In [41]:
check_anchor_url()

-1

In [42]:
def check_request_url():
    a = len(re.findall(r'<(video|img|audio).*?>', content))
    regex_pattern = rf'<(video|img|audio).*?src="(?:https?://)?(?!.*?{re.escape(hostname)}).*?>'
    b = len(re.findall(regex_pattern, content))
    c = b / a if a != 0 else 0

    if c < 0.22:
        return 1
    elif 0.22 <= c <= 0.61:
        return 0
    else:
        return -1

In [43]:
check_request_url()

1

In [48]:
def check_website_links():
    # Finding all the <Meta>, <Script>, and <Link> tags
    meta_tags = soup.find_all('meta')
    script_tags = soup.find_all('script')
    link_tags = soup.find_all('link')
    
    # Counting the total number of links in the tags
    total_links = len(meta_tags) + len(script_tags) + len(link_tags)
    
    # Calculating the percentage of links
    percentage = (total_links / len(soup.find_all())) * 100
    
    # Classifying the website based on the percentage of links
    if percentage < 17:
        return 1
    elif percentage >= 17 and percentage <= 81:
        return 0
    else:
        return -1

def check_anchor_url():
    # Finding all the <a> tags
    anchor_tags = soup.find_all('a')
    
    # Counting the total number of anchor tags
    total_anchors = len(anchor_tags)
    
    # Counting the number of anchor tags with different domain names
    different_domain_count = 0
    for anchor in anchor_tags:
        href = anchor.get('href')
        if href is not None and not re.match(r'^#|javascript:', href):
            if not href.startswith('/') and not href.startswith('http'):
                different_domain_count += 1
    
    # Calculating the percentage of URLs of anchor tags
    percentage = (different_domain_count / total_anchors) * 100
    
    # Classifying the website based on the percentage of URLs of anchor tags
    if percentage < 31:
        return 1
    elif percentage >= 31 and percentage <= 67:
        return 0
    else:
        return -1

def check_request_url():
    # Finding all the external objects (images, videos, sounds) in the webpage
    objects = soup.find_all(['img', 'video', 'audio'])
    
    # Counting the total number of external objects
    total_objects = len(objects)
    
    # Counting the number of external objects with different domains
    different_domain_count = 0
    for obj in objects:
        src = obj.get('src')
        if src is not None:
            parsed_url = urlparse(src)
            if parsed_url.netloc != urlparse(url).netloc:
                different_domain_count += 1
    
    # Calculating the percentage of request URLs for external objects
    percentage = (different_domain_count / total_objects) * 100
    
    # Classifying the webpage based on the percentage of request URLs for external objects
    if percentage < 22:
        return 1
    elif percentage >= 22 and percentage <= 61:
        return 0
    else:
        return -1

def check_email_submission():
    # Checking for the use of "mail()" function in PHP
    if soup.find(lambda tag: tag.name == 'form' and 'mail(' in tag.get('action', '')):
        return -1
    
    # Checking for the use of "mailto:" function
    mailto_links = soup.find_all('a', href=lambda href: href and href.lower().startswith('mailto:'))
    if mailto_links:
        return -1
    
    return 1

def check_different_href_urls():
    # Finding elements with "onMouseOver" attributes
    elements = soup.find_all(attrs={"onMouseOver": True})

    for element in elements:
        # Finding elements with "onMouseOver" and "onClick" attributes
        href_url = element.get("href")
        onclick_href_url = element.get("onClick", "").split("=")[-1].strip("'\";")
        onmouseover_href_url = element.get("onMouseOver", "").split("=")[-1].strip("'\";")

        if href_url:
            if onmouseover_href_url != href_url:
                return 1
        elif onclick_href_url:
            if onmouseover_href_url != onclick_href_url:
                return 1

    return -1

def check_right_click_disabled():
    # Sending a GET request to the URL and retrieving the HTML content
    response = requests.get(url)
    html_content = response.text

    # Searching for the event "event.button==2" in the HTML content
    if ".button==2" in html_content:
        return -1

    return 1

def check_popup_window_text_fields():
    # Finding the URL of the pop-up window
    popup_url = None
    for script in soup.find_all('script'):
        if 'window.open' in script.text:
            popup_url = script.text.split('window.open(')[1].split(',')[0].strip("'")
            break

    if popup_url:
        # Sending a GET request to the pop-up window URL and retrieving the HTML content
        popup_response = requests.get(popup_url)
        popup_html_content = popup_response.text

        # Parsing the pop-up window HTML content
        popup_soup = BeautifulSoup(popup_html_content, 'html.parser')

        # Finding input elements in the pop-up window HTML content
        input_elements = popup_soup.find_all('input')

        for input_element in input_elements:
            input_type = input_element.get('type')
            if input_type == 'text':
                return -1

    return 1

def check_iframe_redirection():
    # Finding iframe elements in the HTML content
    iframe_elements = soup.find_all('iframe')

    if iframe_elements:
        return -1

    return 1

def check_favicon_external_domain():
    # Finding the favicon link element in the HTML content
    favicon_link = soup.find('link', rel=['icon', 'shortcut icon'])

    if favicon_link:
        favicon_url = favicon_link.get('href')
        parsed_url = urlparse(favicon_url)
        domain = parsed_url.netloc

        # Checking if the favicon URL domain is different from the URL domain
        if domain != parsed_url.netloc:
            return -1

    return 1

In [51]:
check_popup_window_text_fields()

1